# Prepare Nearest Neighbour and Road Metrics

The goal of this section is to determine how far each checklist location is from the nearest road, and how far each site is from its nearest neighbour. This involves finding the pairwise distance between a large number of unique checklist locations to a vast number of roads, as well as to each other.

This section is implemented in Python 3, because we want to use `scipy`'s efficient spatial indexing functions based on "kDtrees".

## Prepare libraries

Here, we import required libraries --- you may need to install these for your system.

In [1]:
# import basic python libs
import itertools
from operator import itemgetter
import numpy as np
import matplotlib.pyplot as plt
import math

# libs for dataframes
import pandas as pd

# import libs for geodata
from shapely.ops import nearest_points
import geopandas as gpd

# import ckdtree
from scipy.spatial import cKDTree
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString

## Prepare data for processing

First we read in the roads shapefile, which is obtained from the Open Street Map database.
Then we read in the checklist covariates, and extract the unique coordinate pairs.
All data are reprojected to be in the UTM 43N coordinate system.

We define a custom Python function to separate multi-feature geometries (here, roads which are in parts) into single feature geometries.
Then we define a function to use the K-dimensional trees method from `scipy` to find the distance between two geometries, here, the distance between the locations and the nearest road.
We define another function to find the distance between checklist locations and all other checklist locations.

We use these functions to find the distance between each checklist location and the nearest road and the next nearest site.

## Read in data

Here we read in the data and convert it to a spatial format.

In [4]:
# read in roads shapefile
roads = gpd.read_file("data/spatial/roads_studysite_2019/roads_studysite_2019.shp")
roads.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,22836641,5113,primary,None,SH 100,B,0,0,F,F,"LINESTRING (77.2836076 9.8963634, 77.2840655 9..."
1,22837034,5113,primary,None,SH 100,B,0,0,F,F,"LINESTRING (77.2836076 9.8963634, 77.2843069 9..."
2,22837101,5113,primary,Mettur - Palakkanuthu - Oddanchatram - Palani ...,SH37,B,0,0,F,F,"LINESTRING (77.8362994 10.3714312, 77.83305 10..."
3,22837285,5113,primary,Udumalpet - Dharapuram Road,SH87,B,0,0,F,F,"(LINESTRING (77.2473354 10.5877812, 77.2481291..."
4,28085536,5113,primary,Pollachi - Valaparai Road,SH78,B,0,0,F,F,LINESTRING (76.99275579429218 10.5246287923549...


In [5]:
# read in checklist covariates for conversion to gpd
# get unique coordinates, assign them to the df
# convert df to geo-df
chkCovars = pd.read_csv("data/03_data-covars-perChklist.csv")
unique_locs = chkCovars.drop_duplicates(subset=['longitude',
                                         'latitude'])[['longitude', 'latitude']]
unique_locs['coordId'] = np.arange(1, unique_locs.shape[0]+1)
chkCovars = chkCovars.merge(unique_locs, on=['longitude', 'latitude'])

unique_locs = gpd.GeoDataFrame(
unique_locs, 
geometry = gpd.points_from_xy(unique_locs.longitude, unique_locs.latitude))
unique_locs.crs = {'init' :'epsg:4326'}

# reproject spatials to 43n epsg 32643

roads = roads.to_crs({'init': 'epsg:32643'})
unique_locs = unique_locs.to_crs({'init': 'epsg:32643'})

## Define functions for geometry simplication and distances

In [10]:
# function to simplify multilinestrings
def simplify_roads(complex_roads):
    simpleRoads = []
    for i in range(len(complex_roads.geometry)):
        feature = complex_roads.geometry.iloc[i]
        if feature.geom_type == "LineString":
            simpleRoads.append(feature)
        elif feature.geom_type == "MultiLineString":
            for road_level2 in feature:
                simpleRoads.append(road_level2)
    return simpleRoads

# function to use ckdtrees to find the nearest road
def ckdnearest(gdfA, gdfB):
    A = np.concatenate(
    [np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    simplified_features = simplify_roads(gdfB)
    B = [np.array(geom.coords) for geom in simplified_features]
    B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    return dist

# function to use ckdtrees for nearest other checklist point
def ckdnearest_point(gdfA, gdfB):
    A = np.concatenate(
    [np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    #simplified_features = simplify_roads(gdfB)
    B = np.concatenate(
    [np.array(geom.coords) for geom in gdfB.geometry.to_list()])
    #B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=[2])
    return dist


## Get distance from sites to road and each other

In [11]:
# get distance to nearest road
unique_locs['dist_road'] = ckdnearest(unique_locs, roads)

# get distance to nearest other site
unique_locs['nnb'] = ckdnearest_point(unique_locs, unique_locs)

In [12]:
# check for added columns
unique_locs.head()

,longitude,latitude,coordId,geometry,dist_road,nnb
0,76.702143,10.113145,1,POINT (686508.28186578 1118408.844255172),84.694420,62.056791
4,76.675240,10.114907,2,POINT (683558.60505246 1118588.472478238),12.223700,17.834490
5,76.703636,10.198078,3,POINT (686622.5638861903 1127804.143241186),1525.199913,454.326667
6,76.688492,10.129665,4,POINT (685002.5640544752 1120228.262773657),12.719120,15.374284
7,76.754587,10.127357,5,POINT (692247.8524644603 1120011.292357425),32.038815,126.510762


In [13]:
# write to file
unique_locs = pd.DataFrame(unique_locs.drop(columns='geometry'))
unique_locs['dist_road'] = unique_locs['dist_road']
unique_locs['nnb'] = unique_locs['nnb']
unique_locs.to_csv(path_or_buf = "data/locs_dist_to_road.csv", index=False)

# merge unique locs with chkCovars
chkCovars = chkCovars.merge(unique_locs, on=['latitude', 'longitude', 'coordId'])

In [14]:
# check that metrics have been added to the data
chkCovars.head()

,locality,locality_id,latitude,longitude,observer,sei,duration,distance,area,nObs,decimalTime,julianDate,year,nSp,nSoi,landcover,newjulianDate,coordId,dist_road,nnb
0,Thattekkad--Hornbill Camp,L3203937,10.113145,76.702143,obsr205067,S20847497,90,0.805,NaN,1,12.75,26,2013,9,9,2,57,1,84.69442,62.056791
1,Thattekkad--Hornbill Camp,L3203937,10.113145,76.702143,obsr205067,S21522912,20,0.805,NaN,1,8.00,27,2013,10,10,2,58,1,84.69442,62.056791
2,Thattekkad--Hornbill Camp,L3203937,10.113145,76.702143,obsr205067,S20847495,90,1.609,NaN,1,12.75,24,2013,6,6,2,55,1,84.69442,62.056791
3,Thattekkad--Hornbill Camp,L3203937,10.113145,76.702143,obsr205067,S20847496,90,0.805,NaN,1,12.50,25,2013,4,4,2,56,1,84.69442,62.056791
4,Kalappara,L3314786,10.114907,76.675240,obsr205067,S21522775,90,0.402,NaN,7,6.25,27,2013,7,7,5,58,2,12.22370,17.834490


In [15]:
# save data to local file
# overwrite pre-existing data
chkCovars.to_csv("data/03_data-covars-perChklist.csv")